In [2]:
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
with h5py.File("train_data.h5", 'r') as f:
    X_train = f['X'][:]
    Y_train = f['Y'][:]
    
with h5py.File("val_data.h5", 'r') as f:
    X_val = f['X'][:]
    Y_val = f['Y'][:]
    
with h5py.File("test_data.h5", 'r') as f:
    X_test = f['X'][:]
    Y_test = f['Y'][:]

In [4]:

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(tf.cast(y_true, tf.float32))  # cast to float32 here
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


In [5]:

def build_unet(input_size=(128, 128, 4)):
    inputs = layers.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation('relu')(conv1)
    conv1 = layers.Conv2D(64, (3, 3), padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation('relu')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), padding='same')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation('relu')(conv2)
    conv2 = layers.Conv2D(128, (3, 3), padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation('relu')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), padding='same')(pool2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation('relu')(conv3)
    conv3 = layers.Conv2D(256, (3, 3), padding='same')(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation('relu')(conv3)
    pool3 = layers.MaxPooling2D((2, 2))(conv3)

    # Bottleneck
    conv4 = layers.Conv2D(512, (3, 3), padding='same')(pool3)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation('relu')(conv4)
    conv4 = layers.Conv2D(512, (3, 3), padding='same')(conv4)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation('relu')(conv4)

    # Decoder
    up1 = layers.UpSampling2D((2, 2))(conv4)
    up1 = layers.concatenate([up1, conv3], axis=-1)
    conv5 = layers.Conv2D(256, (3, 3), padding='same')(up1)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.Activation('relu')(conv5)
    conv5 = layers.Conv2D(256, (3, 3), padding='same')(conv5)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.Activation('relu')(conv5)

    up2 = layers.UpSampling2D((2, 2))(conv5)
    up2 = layers.concatenate([up2, conv2], axis=-1)
    conv6 = layers.Conv2D(128, (3, 3), padding='same')(up2)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Activation('relu')(conv6)
    conv6 = layers.Conv2D(128, (3, 3), padding='same')(conv6)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Activation('relu')(conv6)

    up3 = layers.UpSampling2D((2, 2))(conv6)
    up3 = layers.concatenate([up3, conv1], axis=-1)
    conv7 = layers.Conv2D(64, (3, 3), padding='same')(up3)
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Activation('relu')(conv7)
    conv7 = layers.Conv2D(64, (3, 3), padding='same')(conv7)
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Activation('relu')(conv7)

    # Output layer: 1 channel with sigmoid activation for binary mask
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    model = models.Model(inputs, outputs)
    return model

# Build and compile the model
model = build_unet(input_size=(128, 128, 4))
model.compile(optimizer='adam',
              loss=lambda y_true, y_pred: tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred),
              metrics=[dice_coefficient])

model.summary()


W0000 00:00:1747523833.962468    1745 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      2,368 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      1,024 │ conv2d_4[0][0]  

 Total params: 7,794,753 (29.73 MB)

 Trainable params: 7,789,121 (29.71 MB)

 Non-trainable params: 5,632 (22.00 KB)

In [ ]:
# Callbacks for saving best model and early stopping
checkpoint = ModelCheckpoint('Models/brain_tumor_model.keras', monitor='val_dice_coefficient', mode='max', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_dice_coefficient', mode='max', patience=10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_dice_coefficient', mode='max', factor=0.5, patience=5, verbose=1)

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=16,
    epochs=10,
    callbacks=[checkpoint, earlystop, reduce_lr]
)


Epoch 1/10


2025-05-17 23:17:53.925027: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - dice_coefficient: 0.6274 - loss: 0.4029
Epoch 1: val_dice_coefficient improved from -inf to 0.49822, saving model to unet_best.keras


/usr/local/lib/python3.11/dist-packages/keras/src/saving/serialization_lib.py:390: UserWarning: The object being serialized includes a `lambda`. This is unsafe. In order to reload the object, you will have to pass `safe_mode=False` to the loading function. Please avoid using `lambda` in the future, and use named Python functions instead. This is the `lambda` being serialized:               loss=lambda y_true, y_pred: tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred),

  return {key: serialize_keras_object(value) for key, value in obj.items()}


679/679 ━━━━━━━━━━━━━━━━━━━━ 1626s 2s/step - dice_coefficient: 0.6278 - loss: 0.4024 - val_dice_coefficient: 0.4982 - val_loss: 0.3650 - learning_rate: 0.0010
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - dice_coefficient: 1.1507 - loss: -0.2712
Epoch 2: val_dice_coefficient did not improve from 0.49822
679/679 ━━━━━━━━━━━━━━━━━━━━ 1749s 3s/step - dice_coefficient: 1.1507 - loss: -0.2712 - val_dice_coefficient: 0.4838 - val_loss: 0.3536 - learning_rate: 0.0010
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - dice_coefficient: 1.1911 - loss: -0.3240
Epoch 3: val_dice_coefficient improved from 0.49822 to 0.54842, saving model to unet_best.keras
679/679 ━━━━━━━━━━━━━━━━━━━━ 1588s 2s/step - dice_coefficient: 1.1911 - loss: -0.3240 - val_dice_coefficient: 0.5484 - val_loss: 0.2321 - learning_rate: 0.0010
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - dice_coefficient: 1.1981 - loss: -0.3314
Epoch 4: val_dice_coefficient improved from 0.54842 to 0.56247, saving model to unet

In [ ]:
model.save('Models/brain_tumor_model.keras')